In [1]:
import numpy as np
from tqdm import tqdm_notebook as tqdm

class AlternatingLeastSquares():
    def __init__(self, R, k, reg_param, epochs, verbose=False):
        """
        R: rating matrix
        k: latent parameter
        learning_rate: alpha on weight update
        reg_param: beta on weight update
        epochs: training epochs
        verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose
        
    def fit(self):
        # init latent features
        self._users = np.random.normal(size=(self._num_users, self._k))
        self._items = np.random.normal(size=(self._num_items, self._k))
        
        # train while epochs
        self._training_process = []
        self._user_error = 0; self._item_error = 0;
        for epoch in range(self._epochs):
            # user latent space에 대해 업데이트
            for i, Ri in enumerate(self._R):
                self._users[i] = self.user_latent(i, Ri)
                self._user_error = self.cost()
                
            # item latent space에 대해 업데이트
            for j, Rj in enumerate(self._R.T):
                self._items[j] = self.item_latent(j, Rj)
                self._item_error = self.cost()
                
            cost = self.cost()
            self._training_process.append((epoch, cost))
            
            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print('Iteration: %d ; cost = %.4f' % (epoch + 1, cost))
                
    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        xi, yi = self._R.nonzero()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost/len(xi))
    
    def user_latent(self, i, Ri):
        """
        error: rating - prediction error
        i: user index
        Ri: rating of user index i
        return: convergence value of user latent of i index
        """
        
        du = np.linalg.solve(np.dot(self._items.T, np.dot(np.diag(Ri), self._items)) + self._reg_param * np.eye(self._k),
                             np.dot(self._items.T, np.dot(np.diag(Ri), self._R[i].T))).T
        return du
    
    def item_latent(self, j, Rj):
        """
        error: rating - prediction error
        j: item index
        Rj: rating of item index i
        return: convergence value of item latent of j index
        """
        
        di = np.linalg.solve(np.dot(self._users.T, np.dot(np.diag(Rj), self._users)) + self._reg_param * np.eye(self._k),
                             np.dot(self._users.T, np.dot(np.diag(Rj), self._R[:, j])))
        return di
    
    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._users[i, :].dot(self._items[j, :].T)
    
    def get_complete_matrix(self):
        """
        return: complete matrix R^
        """
        return self._users.dot(self._items.T)
    
# run example
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5)
    R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0],
    ])

ALS는 하나를 고정시키고 훈련시키는 것에 유의한다!

In [2]:
als = AlternatingLeastSquares(R = R, reg_param=0.01, epochs=100, 
                              verbose=True, k=3)
als.fit()

Iteration: 10 ; cost = 0.0082
Iteration: 20 ; cost = 0.0048
Iteration: 30 ; cost = 0.0036
Iteration: 40 ; cost = 0.0032
Iteration: 50 ; cost = 0.0031
Iteration: 60 ; cost = 0.0031
Iteration: 70 ; cost = 0.0031
Iteration: 80 ; cost = 0.0032
Iteration: 90 ; cost = 0.0032
Iteration: 100 ; cost = 0.0033


In [3]:
als.get_complete_matrix()

array([[ 0.99539598, -0.10731879,  4.8104796 ,  0.99404627,  2.99567876],
       [ 2.0032064 ,  0.0753728 ,  2.99479893,  0.99958361,  1.00793668],
       [ 0.99752738,  1.99881524, -1.24935503,  4.99855531, -1.64020287],
       [ 0.99769522,  0.88031259,  6.21488851,  4.00084823,  4.00097861],
       [ 2.00506203,  1.00083417,  4.99986873,  4.0003238 ,  2.46620236],
       [ 4.99822212,  0.99918924,  5.00098269,  3.99972969,  0.62698713],
       [ 0.47694037,  0.37834951, -0.04641601,  0.99703439, -0.34524925]])